In [1]:
import pandas as pd
df = pd.read_csv('SampleDataset.csv', encoding='latin-1')

In [2]:
df.head()

,Sno,TECHNOLOGY,ORIGINAL/SYNONYM,QUESTION,SOLUTION
0,1,PowerBI,Original,Issue with Power bi date slicer,If you are experiencing an issue with the date...
1,2,PowerBI,Synonym,What are some common issues that can arise wit...,same as above
2,3,PowerBI,Synonym,How do I resolve issues with the date slicer i...,Same as above
3,4,PowerBI,Synonym,I'm having trouble getting the date slicer in ...,Same as above
4,5,PowerBI,Synonym,What could be causing the date slicer in Power...,Same as above


In [3]:
df['Sentence'] = df['TECHNOLOGY'] + " - " + df['QUESTION']
df.head()

,Sno,TECHNOLOGY,ORIGINAL/SYNONYM,QUESTION,SOLUTION,Sentence
0,1,PowerBI,Original,Issue with Power bi date slicer,If you are experiencing an issue with the date...,PowerBI - Issue with Power bi date slicer
1,2,PowerBI,Synonym,What are some common issues that can arise wit...,same as above,PowerBI - What are some common issues that can...
2,3,PowerBI,Synonym,How do I resolve issues with the date slicer i...,Same as above,PowerBI - How do I resolve issues with the dat...
3,4,PowerBI,Synonym,I'm having trouble getting the date slicer in ...,Same as above,PowerBI - I'm having trouble getting the date ...
4,5,PowerBI,Synonym,What could be causing the date slicer in Power...,Same as above,PowerBI - What could be causing the date slice...


In [4]:
sentences = []
for row in df['Sentence']:
    sentences.append(row)
sentences[1:8]

['PowerBI - What are some common issues that can arise with the date slicer in Power BI, and how can I troubleshoot them?',
 'PowerBI - How do I resolve issues with the date slicer in Power BI when it is not \nfiltering data correctly or not displaying all available dates?',
 "PowerBI - I'm having trouble getting the date slicer in Power BI to work properly. Are \nthere any specific settings or configurations I should check to resolve the issue?",
 'PowerBI - What could be causing the date slicer in Power BI to malfunction, and what \nsteps can I take to fix it?',
 'PowerBI - Can you provide any guidance on troubleshooting problems with the date \nslicer in Power BI, particularly when it comes to issues with filtering or interaction with other visuals and filters?',
 'PowerBI - The date slicer is not filtering data correctly',
 'PowerBI - What should I do if my Power BI date slicer is not filtering data correctly?']

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[-0.05455767  0.07401682  0.00639783 ... -0.04954586 -0.05768156
  -0.02505917]
 [-0.04970558  0.03084414  0.01144504 ... -0.06109059 -0.07576088
  -0.01194993]
 [-0.00439032  0.06759153  0.00364703 ... -0.05403583 -0.07599875
   0.00057956]
 ...
 [-0.01956023 -0.06185008 -0.05478382 ... -0.03913626  0.02141256
  -0.06274449]
 [-0.00899348 -0.08685958 -0.05075125 ... -0.04794437  0.02407051
  -0.07409859]
 [-0.03058355 -0.07275829 -0.05993147 ... -0.06807294  0.05030226
  -0.06599381]]


In [6]:
print(len(embeddings))
print(len(embeddings[0]))

360
384


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(embeddings[0].reshape(1,-1),embeddings[1].reshape(1,-1)))
print(cosine_similarity(embeddings[0].reshape(1,-1),embeddings[102].reshape(1,-1)))

[[0.91687393]]
[[0.19741663]]


In [8]:
answer = []
for index, val in enumerate(df['ORIGINAL/SYNONYM']):
    if str(val) == "Original":
        answer.append(index)

In [9]:
text = input("Enter a sentence: ")

Enter a sentence: PowerBI - I am trying to create a dashboard. I used date slicer to filter dates to reflect data on the table. There are data in table till 2022 but the data slicer is moved beyond 4/31/2021 is not showing data in the tables


In [10]:
text_embedding = model.encode(text)

In [11]:
ind = 0
sim = 0
for index, val in enumerate(embeddings):
    res = cosine_similarity(text_embedding.reshape(1,-1),embeddings[index].reshape(1,-1))
    if res[0][0] > sim:
        sim = res[0][0]
        ind = index
print("The Cosine Similarity between the Sentences is:", sim)

The Cosine Similarity between the Sentences is: 0.77476656


In [12]:
for i in range(len(answer)):
    if answer[i] > ind:
        print(df['SOLUTION'][answer[i - 1]])
        break

If you are experiencing an issue with the date slicer in Power BI, there could be several reasons why this is happening. Here are some common issues and their potential solutions:
1. The date slicer is not filtering data correctly: This could be due to incorrect data type or formatting of the date column in the data source. To resolve this issue, check the data type of the date column and ensure that it matches the format of the date slicer.
2. The date slicer is not displaying all the available dates: This could be due to the range of dates selected in the slicer. To resolve this issue, adjust the range of dates displayed in the slicer or increase the granularity of the slicer.
3. The date slicer is not working at all: This could be due to a corrupted Power BI file or a bug in the Power BI software. To resolve this issue, try restarting Power BI or creating a new file and recreating the date slicer.
4. The date slicer is not interacting with other visuals or filters: This could be due